# DECK BUILDER

This model should be able to create a deck based on certain cards chosen by the user. If, for instance, a user has a deck that they would like to use, but is missing a card, they should be able to input a list of cards and the model will help choose the best card based on that list.

In [6]:
import pandas as pd
import numpy as np
import ast
import os

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

from cards import *
from accuracy import *

# Data Loading and Prep

In [2]:
cards_master = pd.read_csv('data\CardMasterListSeason18_12082020.csv')

In [3]:
battle_chunks = pd.read_csv("data\BattlesStaging_01022021_WL_tagged.csv", chunksize=50000)

chunks_list = []

for chunk in battle_chunks:
    chunk = chunk[(chunk["arena.id"] == 54000050.0) & (chunk["gameMode.id"] == 72000201.0) ]
    chunk = chunk[['winner.cards.list', 'loser.cards.list']]
    chunks_list.append(chunk)

battles = pd.concat(chunks_list, ignore_index=True)

In [4]:
def convert_string_to_list(deck_string):
    try:
        deck = ast.literal_eval(deck_string)
        if isinstance(deck, list):
            return deck
        else:
            raise ValueError("The input is not a list.")
    except:
        raise ValueError("Error in converting string to list.")

In [ ]:
winner_list = battles['winner.cards.list'].apply(convert_string_to_list)
loser_list = battles['loser.cards.list'].apply(convert_string_to_list)   

In [31]:
one_hot_encoded = pd.get_dummies(cards_master, columns=['team.card1.id'], prefix='', prefix_sep='')
one_hot_encoded = one_hot_encoded.drop(columns='team.card1.name')

In [30]:
def create_deck_dataframe(winner_list, one_hot_encoded, cards_master):

    deck_df = pd.DataFrame(0, index=range(len(winner_list)), columns=one_hot_encoded.columns)

    for i, deck in enumerate(winner_list):
        for card_id in deck:
            col_name = str(card_id)
            if col_name in deck_df.columns:
                deck_df.at[i, col_name] = 1

    id_to_name_map = dict(zip(cards_master['team.card1.id'].astype(str), 'Winner' + cards_master['team.card1.name']))
    deck_df.rename(columns=id_to_name_map, inplace=True)

    return deck_df

In [ ]:
winners_ohe = create_deck_dataframe(winner_list, one_hot_encoded, cards_master)
losers_ohe = create_deck_dataframe(loser_list, one_hot_encoded, cards_master)   

In [ ]:
winners_ohe

In [ ]:
winners_ohe.to_csv('data/winners_ohe_jan2')
losers_ohe.to_csv('data/losers_ohe_jan2')

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
    
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]

 
        batch_x = np.array(batch_x).reshape((-1, 102))  

     
        return batch_x, batch_x

batch_size = 32 
x_gen = DataGenerator(winners_ohe, batch_size)
y_gen = DataGenerator(losers_ohe, batch_size)

In [ ]:
#Checking GPU availability
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4504224062675142078
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3655335936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11153467939096673049
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [ ]:
num_cards = 102

deck_matrix = winners_ohe

input_layer = Input(shape=(num_cards, ))
x = Dense(128, activation='relu')(input_layer)
output_layer = Dense(num_cards, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy')

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 102)]             0         
                                                                 
 dense_2 (Dense)             (None, 128)               13184     
                                                                 
 dense_3 (Dense)             (None, 102)               13158     
                                                                 
Total params: 26,342
Trainable params: 26,342
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_gen, epochs=10, validation_data=y_gen)

Epoch 1/10
87483/87483 [==============================] - 233s 3ms/step - loss: 0.0014 - val_loss: 1.5166e-08
Epoch 2/10
87483/87483 [==============================] - 227s 3ms/step - loss: 6.8274e-09 - val_loss: 6.8299e-09
Epoch 3/10
87483/87483 [==============================] - 226s 3ms/step - loss: 3.7607e-09 - val_loss: 4.4927e-09
Epoch 4/10
87483/87483 [==============================] - 226s 3ms/step - loss: 2.6624e-09 - val_loss: 3.3748e-09
Epoch 5/10
87483/87483 [==============================] - 234s 3ms/step - loss: 2.0898e-09 - val_loss: 2.8089e-09
Epoch 6/10
87483/87483 [==============================] - 225s 3ms/step - loss: 1.7362e-09 - val_loss: 2.3751e-09
Epoch 7/10
87483/87483 [==============================] - 234s 3ms/step - loss: 1.4961e-09 - val_loss: 2.0570e-09
Epoch 8/10
87483/87483 [==============================] - 226s 3ms/step - loss: 1.3208e-09 - val_loss: 1.8464e-09
Epoch 9/10
87483/87483 [==============================] - 225s 3ms/step - loss: 1.1884e-09 -

In [ ]:
model.save('models\deck_builder.h5')

In [ ]:
def deck_builder(unfinished):
  cards = pd.read_csv('data\CardMasterListSeason18_12082020.csv')['team.card1.name'].to_list()
  selection = []
  for card in cards:
    if card in unfinished:
      selection += [1]
    else:
      selection += [0]
  vec = np.array(selection).reshape(1, -1)
  probabilities = model.predict(vec)
  top_8_indices = np.argsort(probabilities[0])[-8:]

  lst = np.ones(102)
  for i in range(102):
    if i in top_8_indices:
      lst[i] = 0

  thing = abs(lst - 1).astype(bool)
  lstcards = pd.Series(cards)
  return lstcards[thing]


In [ ]:
deck_builder(["Knight", "Lumberjack", "Giant"])

1/1 [==============================] - 0s 107ms/step


0             Knight
3              Giant
5            Minions
7              Witch
17            Wizard
35        Lumberjack
69    Electro Spirit
86            Rocket
dtype: object

# COUNTER PREDICTOR

In [ ]:
num_cards = 102

counter_model = Sequential()
counter_model.add(Dense(128, activation='relu', input_shape=(num_cards,)))
counter_model.add(Dense(64, activation='relu'))
counter_model.add(Dense(32, activation='relu'))
counter_model.add(Dense(num_cards, activation='sigmoid'))

counter_model.compile(optimizer='adam', loss='binary_crossentropy')

counter_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               13184     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 102)               3366      
                                                                 
Total params: 26,886
Trainable params: 26,886
Non-trainable params: 0
_________________________________________________________________


In [7]:
losers_ohe = pd.read_csv('data/losers_ohe_jan2')
winners_ohe = pd.read_csv('data/winners_ohe_jan2')

In [22]:
losers_ohe.iloc[:,1:]

,WinnerKnight,WinnerArchers,WinnerGoblins,WinnerGiant,WinnerP.E.K.K.A,WinnerMinions,WinnerBalloon,WinnerWitch,WinnerBarbarians,WinnerGolem,...,WinnerPoison,WinnerGraveyard,WinnerThe Log,WinnerTornado,WinnerClone,WinnerEarthquake,WinnerBarbarian Barrel,WinnerHeal Spirit,WinnerGiant Snowball,WinnerRoyal Delivery
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2799420,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2799421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2799422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2799423,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array(batch_x), np.array(batch_y)

In [ ]:
data_gen = DataGenerator(losers_ohe, winners_ohe, batch_size=32)

In [ ]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6221886355370302969
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3655335936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17810559033226479973
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [ ]:
counter_model.fit(data_gen, epochs=7)

Epoch 1/7


87483/87483 [==============================] - 154s 2ms/step - loss: 0.2446
Epoch 2/7
87483/87483 [==============================] - 150s 2ms/step - loss: 0.2445
Epoch 3/7
87483/87483 [==============================] - 147s 2ms/step - loss: 0.2445
Epoch 4/7
87483/87483 [==============================] - 149s 2ms/step - loss: 0.2445
Epoch 5/7
87483/87483 [==============================] - 147s 2ms/step - loss: 0.2444
Epoch 6/7
87483/87483 [==============================] - 156s 2ms/step - loss: 0.2444
Epoch 7/7
87483/87483 [==============================] - 172s 2ms/step - loss: 0.2444


In [ ]:
counter_model.save("models\counter_model.h5")

In [ ]:
x = np.array([[1,1,1,1,1,1,1,1] + [0 for i in range(102-8)]])
counter_model.predict(x)

1/1 [==============================] - 1s 1s/step


array([[0.50665504, 0.48609108, 0.48651543, 0.5096583 , 0.47931036,
        0.47179055, 0.521534  , 0.48437423, 0.54247093, 0.4950773 ,
        0.53707075, 0.4860205 , 0.4876652 , 0.47261554, 0.5325402 ,
        0.4932935 , 0.51606923, 0.52157867, 0.54113215, 0.49461028,
        0.53459513, 0.50282526, 0.519653  , 0.49185497, 0.50661576,
        0.4684379 , 0.52902156, 0.44615385, 0.4963718 , 0.50844985,
        0.5028626 , 0.5013515 , 0.4982493 , 0.5154622 , 0.5210462 ,
        0.5093011 , 0.52344817, 0.49101165, 0.48382252, 0.5314757 ,
        0.50643843, 0.4662065 , 0.50587493, 0.52178246, 0.48961002,
        0.53961563, 0.49779013, 0.45869938, 0.47083747, 0.5133197 ,
        0.48484603, 0.5165    , 0.5172466 , 0.51174057, 0.45540622,
        0.51840675, 0.4750868 , 0.49225685, 0.5106354 , 0.51076907,
        0.47438335, 0.5105532 , 0.49959013, 0.5113279 , 0.50864255,
        0.5023642 , 0.4950939 , 0.4735829 , 0.49714488, 0.47066718,
        0.46888265, 0.49351993, 0.47223693, 0.48

In [ ]:
def counter_predictor(opponent_deck):
  cards = pd.read_csv('data\CardMasterListSeason18_12082020.csv')['team.card1.name'].to_list()
  selection = []
  for card in cards:
    if card in opponent_deck:
      selection += [1]
    else:
      selection += [0]
  vec = np.array(selection).reshape(1, -1)
  probabilities = counter_model.predict(vec)
  top_8_indices = np.argsort(probabilities[0])[-8:]

  lst = np.ones(102)
  for i in range(102):
    if i in top_8_indices:
      lst[i] = 0

  thing = abs(lst - 1).astype(bool)
  lstcards = pd.Series(cards)
  return lstcards[thing]

In [ ]:
counter_predictor(["Witch", "Skeleton Army", "Musketeer", "Hog Rider", "Freeze", "Valkyrie", "Knight", "Inferno Tower"])

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 16ms/step


11         Valkyrie
12    Skeleton Army
17           Wizard
21        Hog Rider
83         Fireball
84           Arrows
91              Zap
94          The Log
dtype: object

# Train Test Accuracy

In [25]:
def predict_probabilities_in_batches(model, dataframe, batch_size=1024):
    num_samples = len(dataframe)
    predicted = []
    
    for start in range(0, num_samples, batch_size):
        end = min(start + batch_size, num_samples)
        batch = dataframe.iloc[start:end]
        batch_predictions = model.predict(batch)
        predicted.append(batch_predictions)

    return np.vstack(predicted)

In [23]:
counter_model = load_model('models/counter_model.h5')

In [26]:
train_predicted_probabilities = predict_probabilities_in_batches(counter_model, losers_ohe)

27/27 [==============================] - 0s 848us/step


In [27]:
train_accuracy = custom_accuracy(winners_ohe, train_predicted_probabilities, card_to_type)
train_accuracy

0.7068478943581231

In [28]:
test_set = pd.read_csv('data\REDUCED_JAN1_BATTLES.csv')

In [32]:
test_winner = test_set['winner.cards.list'].apply(convert_string_to_list)
test_winners_ohe = create_deck_dataframe(test_winner, one_hot_encoded, cards_master)
                                                                                                                
test_loser = test_set['loser.cards.list'].apply(convert_string_to_list)
test_loser_ohe = create_deck_dataframe(test_loser, one_hot_encoded, cards_master)

In [34]:
test_predicted_probabilities = predict_probabilities_in_batches(counter_model, test_loser_ohe)

11/11 [==============================] - 0s 801us/step


In [36]:
custom_accuracy(test_winners_ohe, test_predicted_probabilities, card_to_type)
#test_accuracy

0.7006404815363316